![Scala Intro](../misc/thumbnail.jpg)

In [56]:
interp.configureCompiler(_.settings.nowarn.value = false)

## Session Objectives

1. Grasp the **fundamentals of FP** by means of Scala
2. Learn the basics of **algebraic data types** (ADTs)
3. Understand the implications of having **functions as first-class citizens**
4. Get used to the **syntax** that simplifies dealing with functions

## So, What is Functional Programming?

Functional Programming is **"Programming With Pure Functions"**

([According to Wikipedia](https://en.wikipedia.org/wiki/Pure_function)) A pure function is a function that has the following properties:
1. Its return value is the same for the same arguments
2. Its evaluation has no side effects

#### Pure function example

In [57]:
def pure(x: Int, y: Int): Int = x + y

defined function pure

In [58]:
pure(1, 2)
pure(1, 2)

res57_0: Int = 3
res57_1: Int = 3

#### Impure function example

In [59]:
var acc: Int = 0
def impure(x: Int, y: Int): Int = {
    acc += 1
    x + y + acc
}

acc: Int = 2
defined function impure

In [60]:
impure(1, 2)
impure(1, 2)

res59_0: Int = 4
res59_1: Int = 5

#### Immutability

In [60]:
val x: Int = 0
x = 2

cmd60.sc:2: reassignment to val
val res60_1 = x = 2
                ^Compilation Failed

: 

#### [Why Functional Programming Matters?](https://www.cs.kent.ac.uk/people/staff/dat/miranda/whyfp90.pdf) (John Hughes)

> *"We conclude that since ***modularity*** is the key to successful programming, functional programming offers important advantages for software development."*

![Indiana Jones](../misc/grial.jpg)

## Algebraic Data Types

* Make a data type to work with int lists

In [ ]:
sealed trait IList
class End() extends IList
class Cons(head: Int, tail: IList) extends IList

* We create a method `prepend` to add a new element in the front of the list

In [61]:
sealed trait IList {
    def prepend(h: Int): IList = new Cons(h, this)
}
class End() extends IList
class Cons(head: Int, tail: IList) extends IList

defined trait IList
defined class End
defined class Cons

In [62]:
val xs: IList = new Cons(1, new Cons(2, new End())) // IList(1, 2)
xs.prepend(0) // IList(0, 1, 2)

xs: IList = ammonite.$sess.cmd60$Helper$Cons@834c59c
res61_1: IList = ammonite.$sess.cmd60$Helper$Cons@3132aab4

### Case classes

* They provide common utilities automatically: shorter constructor, `toString` method, `copy` method, etc.

In [63]:
case class Bicicleta(cadencia: Int, marcha: Int, velocidad: Int)

defined class Bicicleta

In [64]:
new Bicicleta(1, 2, 3)
val b = Bicicleta(1, 2, 3)
b.cadencia
b.velocidad
b.copy(velocidad = 4)

res63_0: Bicicleta = Bicicleta(1, 2, 3)
b: Bicicleta = Bicicleta(1, 2, 3)
res63_2: Int = 1
res63_3: Int = 3
res63_4: Bicicleta = Bicicleta(1, 2, 4)

* We can use it to make list creation simpler and to enable reading instance contents

In [66]:
sealed trait IList {
    def prepend(h: Int): IList = new Cons(h, this)
}
case class End() extends IList
case class Cons(head: Int, tail: IList) extends IList

defined trait IList
defined class End
defined class Cons

In [67]:
val xs: IList = Cons(1, Cons(2, End()))
xs.prepend(0)

xs: IList = Cons(1, Cons(2, End()))
res66_1: IList = Cons(0, Cons(1, Cons(2, End())))

### Pattern Matching

* We can use this technique to compare a value against a sequence of patterns

In [68]:
def myPm(x: Any): Int = x match {
    case i: Int => i
    case s: String => s.length
    case _ => -1
}

defined function myPm

In [69]:
myPm(5)
myPm("hi")
myPm(3.0)

res68_0: Int = 5
res68_1: Int = 2
res68_2: Int = -1

* Pattern matching turns out to be really useful in combination with case classes (`sum` method)

In [73]:
sealed trait IList {
    def prepend(h: Int): IList = new Cons(h, this)
    def sum: Int = this match {
        case End() => 0
        case Cons(h, t) => h + t.sum
    }
}
case class End() extends IList
case class Cons(head: Int, tail: IList) extends IList

defined trait IList
defined class End
defined class Cons

In [71]:
Cons(1, Cons(2, Cons(3, End()))).sum

res70: Int = 6

* Invocation of `sum` method

## Lambda Expressions

* So far we've been working with methods (`incr` example)

In [74]:
def incr(x: Int): Int = x + 1
incr(0)

defined function incr
res73_1: Int = 1

* But functions are *first-class citizens* (ugly lambda expressions)

In [76]:
val x: Int =0
val s: String = "hola"
val f: Function1[Int, Int] = new Function1[Int, Int] {
    def apply(x: Int): Int = x + 1
}

x: Int = 0
s: String = "hola"
f: Int => Int = <function1>

In [77]:
f.apply(0)

res76: Int = 1

* We provide a new method, which `maps`s every element in the list

In [78]:
sealed trait IList {
    def prepend(h: Int): IList = new Cons(h, this)
    def sum: Int = this match {
        case End() => 0
        case Cons(h, t) => h + t.sum
    }
    def map(f: Function1[Int, Int]): IList = this match {
        case End() => End()
        case Cons(h, t) => Cons(f(h), t.map(f))
    }
}
case class End() extends IList
case class Cons(head: Int, tail: IList) extends IList

defined trait IList
defined class End
defined class Cons

In [79]:
val xs: IList = Cons(1, Cons(2, Cons(3, End())))
xs.map(f)

xs: IList = Cons(1, Cons(2, Cons(3, End())))
res78_1: IList = Cons(2, Cons(3, Cons(4, End())))

## Syntactic Sugar

![azucar](../misc/azucar.jpg)

* An operator is just a method (`concatenate` & `prepend`)

In [82]:
sealed trait IList {
    def ::(h: Int): IList = new Cons(h, this)
    def sum: Int = this match {
        case End() => 0
        case Cons(h, t) => h + t.sum
    }
    def map(f: Function1[Int, Int]): IList = this match {
        case End() => End()
        case Cons(h, t) => Cons(f(h), t.map(f))
    }
    def ++(other: IList): IList = this match {
        case End() => other
        case Cons(h, t) => Cons(h, t ++ other)
    }
}
case class End() extends IList
case class Cons(head: Int, tail: IList) extends IList

defined trait IList
defined class End
defined class Cons

In [83]:
val xs: IList = Cons(1, Cons(2, End()))
xs ++ xs
xs.::(0)
0 :: xs

xs: IList = Cons(1, Cons(2, End()))
res82_1: IList = Cons(1, Cons(2, Cons(1, Cons(2, End()))))
res82_2: IList = Cons(0, Cons(1, Cons(2, End())))
res82_3: IList = Cons(0, Cons(1, Cons(2, End())))

* It's possible to declare default arguments for parameters

In [84]:
sealed trait IList {
    def ::(h: Int): IList = new Cons(h, this)
    def sum: Int = this match {
        case End() => 0
        case Cons(h, t) => h + t.sum
    }
    def map(f: Function1[Int, Int]): IList = this match {
        case End() => End()
        case Cons(h, t) => Cons(f(h), t.map(f))
    }
    def ++(other: IList): IList = this match {
        case End() => other
        case Cons(h, t) => Cons(h, t ++ other)
    }
}
case class End() extends IList
case class Cons(head: Int, tail: IList = End()) extends IList

defined trait IList
defined class End
defined class Cons

In [85]:
Cons(1, End())
Cons(1)

res84_0: Cons = Cons(1, End())
res84_1: Cons = Cons(1, End())

* Variadic methods are great for certain situations like creating lists

In [87]:
object IList {
    def crear(xs: Int*): IList =
      if (xs.isEmpty) End() else Cons(xs.head, crear(xs.tail:_*))
}

defined object IList

In [89]:
IList.crear(1, 2, 3)
IList.crear()
IList.crear(1,2,3,4,5)

res88_0: IList = Cons(1, Cons(2, Cons(3, End())))
res88_1: IList = End()
res88_2: IList = Cons(1, Cons(2, Cons(3, Cons(4, Cons(5, End())))))

* If you use `apply` as a method name, it's applied automatically

In [90]:
object IList {
    def apply(xs: Int*): IList =
      if (xs.isEmpty) End() else Cons(xs.head, apply(xs.tail:_*))
}

defined object IList

In [91]:
IList.apply(1, 2, 3)
IList(1, 2, 3)

res90_0: IList = Cons(1, Cons(2, Cons(3, End())))
res90_1: IList = Cons(1, Cons(2, Cons(3, End())))

* Scala deploys syntax to simplify the creation of Lambda Expressions 

In [92]:
val incr: Int => Int = (i: Int) => i + 1

incr: Int => Int = ammonite.$sess.cmd91$Helper$$Lambda$3264/201197253@e8b4e09

In [94]:
incr(0)

res93: Int = 1

In [95]:
val incr: Int => Int = i => i + 1

incr: Int => Int = ammonite.$sess.cmd94$Helper$$Lambda$3272/1627399875@3d7daf5f

In [97]:
val incr: Int => Int = _ + 1
incr(0)

incr: Int => Int = ammonite.$sess.cmd96$Helper$$Lambda$3282/416076863@4bbca63c
res96_1: Int = 1

In [98]:
IList(1, 2, 3).map(_ + 1)
IList(1, 2, 3).map(_ * 5)

res97_0: IList = Cons(2, Cons(3, Cons(4, End())))
res97_1: IList = Cons(5, Cons(10, Cons(15, End())))

## More ADT examples

* Optional Int

In [99]:
sealed trait IOption
case class INone() extends IOption
case class ISome(i: Int) extends IOption

defined trait IOption
defined class INone
defined class ISome

* Either Error or Int

In [100]:
sealed trait IEither
case class Left(err: Error) extends IEither
case class Right(i: Int) extends IEither

defined trait IEither
defined class Left
defined class Right

## Final Result

In [1]:
// IList is an Algebraic Data Type, made of Cons and End
sealed trait IList {
    
  // Operators are available
  def ::(h: Int): IList = Cons(h, this)
    
  // Pattern matching is very handy to deal with ADTs
  def sum: Int = this match {
    case Cons(h, t) => h + t.sum
    case End() => 0
  }
    
  // Functions can be passed as parameters
  // `map` is a higher order function
  def map(f: Int => Int): IList = this match {
    case Cons(h, t) => Cons(f(h), t.map(f))
    case End() => End()
  }
    
  def ++(other: IList): IList = this match {
    case Cons(h, t) => Cons(h, t ++ other)
    case End() => other
  }
}

// Case classes mitigate common boilerplate
// Default parameters could be helpful
case class Cons(val head: Int, val tail: IList = End()) extends IList
case class End() extends IList

object IList {
  // Variadic arguments are idoneous as a list creator
  def apply(xs: Int*): IList = {
    if (xs.isEmpty) End()
    else Cons(xs.head, apply(xs.tail:_*)) 
  }
}

defined class IList
defined class Cons
defined class End
defined object IList

## ADTs & Dotty

```scala
enum IList:

  case Cons(head: Int, tail: IList)
  case End()

  def sum: Int = this match
    case Cons(h, t) => h + t.sum
    case End() => 0

  def map(f: Int => Int): IList = this match
    case Cons(h, t) => Cons(f(h), t.map(f))
    case End() => End()

  def ++(other: IList): IList = this match
    case Cons(h, t) => Cons(h, t ++ other)
    case End() => other

object IList:
  def apply(is: Int*): IList =
    if (is.isEmpty) End() else Cons(is.head, apply(is.tail:_*))
```

## Takeaways

* Functional programming is programming with pure functions
* Algebraic data types are encoded as a "sum" of case clases
* Functions are treated as first-class citizens, which enables *higher order functions*
* Syntactic sugar is convenient to dulcify expressions
* Dotty has introduced many features towards the functional side
* This is just the beginning: *type classes*, *DSLs*, *generic programming*, etc.

In [101]:
println("Thank you!")

Thank you!


![Applause](../misc/applause.gif)